In [0]:
catalog = "movie"
b_schema = "movie_bronze"
s_schema = "movie_silver"
g_schema = "movie_gold"

In [0]:
spark.sql(f"""
SELECT 
  CONCAT('G000', movie.ID) as movie_id,
  movie.Title as movie_title,
  movie.Year as release_year,
  NVL(movie.Rating, 'not rated') as rating,
  CASE 
    WHEN TRY_CAST(movie.IMDb AS DOUBLE) IS NULL OR isnan(TRY_CAST(movie.IMDb AS DOUBLE)) THEN '0'
    ELSE LTRIM(RTRIM(movie.IMDb))
  END as IMDb_rating,
  CASE WHEN movie.`Rotten Tomatoes` = 'na' THEN '0' ELSE LTRIM(RTRIM(NVL(movie.`Rotten Tomatoes`, '0'))) END as Rotten_Tomatoes_rating,
  (
    SELECT master.Genre_id 
    FROM movie.movie_silver.movie_genre_master master 
    WHERE movie.Genre = master.Genre 
    LIMIT 1
  ) as Genre_id,
  (
    CASE 
      WHEN movie.Netflix = 1 THEN (
        SELECT platform_id 
        FROM movie.movie_silver.movie_platform_master 
        WHERE platform_name = 'Netflix' 
        LIMIT 1
      )
      WHEN movie.`Amazon Prime Video` = 1 THEN (
        SELECT platform_id 
        FROM movie.movie_silver.movie_platform_master 
        WHERE platform_name = 'Amazon Prime Video' 
        LIMIT 1
      )
      ELSE 'Other' 
    END
  ) as platform_id
FROM movie.movie_bronze.movies movie""").createOrReplaceTempView("movie_temp")

In [0]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS {catalog}.{s_schema}.movie_data_transaction (
    movie_id STRING,
    movie_title STRING,
    release_year INT,
    rating STRING,
    IMDb_rating STRING,
    Rotten_Tomatoes_rating STRING,
    Genre_id STRING,
    platform_id STRING,

    PRIMARY KEY (movie_id),
    FOREIGN KEY (Genre_id) REFERENCES {catalog}.{s_schema}.movie_genre_master(Genre_id),
    FOREIGN KEY (platform_id) REFERENCES {catalog}.{s_schema}.movie_platform_master(platform_id)) USING DELTA""").display()

In [0]:
%sql
TRUNCATE TABLE movie.movie_silver.movie_data_transaction ;
INSERT INTO movie.movie_silver.movie_data_transaction  SELECT * FROM movie_temp;

In [0]:
%sql
SELECT * FROM movie.movie_silver.movie_data_transaction